In [1]:
import numpy as np
import pandas as pd
from openpyxl import load_workbook

In [3]:
# 1) devi prima estrarre manualmenete tuutt i files dall'archivio 7z
# 2) poi cambiare in questo codice il percorso del file estretti e il loro nome
df_cpu=pd.read_csv('C:\\Users\\ccarbon\\OneDrive - Vodafone Group\\JOBFolder\\CSCF_Dashboard\\UK\\Jul21\\Jul21_UK_KPI CPU (raw).csv')
df_bhca=pd.read_csv('C:\\Users\\ccarbon\\OneDrive - Vodafone Group\\JOBFolder\\CSCF_Dashboard\\UK\\Jul21\\Jul21_UK_KPI BHCA (raw).csv')
df_reg=pd.read_csv('C:\\Users\\ccarbon\\OneDrive - Vodafone Group\\JOBFolder\\CSCF_Dashboard\\UK\\Jul21\\Jul21_UK_CSCF Sess+Reg raw (1 month).csv')
mese=input('immetterte il numero mese [es. 1 per Gennaio]>')

immetterte il numero mese [es. 1 per Gennaio]>7


In [4]:
#elaborazione per df_cpu
day=df_cpu['Hour of DateTime'].str.split(" ").str.get(0)
month=df_cpu['Hour of DateTime'].str.split(" ").str.get(1)
year=df_cpu['Hour of DateTime'].str.split(" ").str.get(2)
df_cpu['hour']=df_cpu['Hour of DateTime'].str.split(" ").str.get(3)
d = {'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May':'5', 'June':'6', 'July':'7','August':'8','September':'9','October':'10','November':'11','December':'12'}
month = month.map(d)
df_cpu['month']=month
df_cpu['date']=day.str.cat(month, sep='/').str.cat(year, sep='/')
df_cpu=df_cpu[['date', 'hour','CSCF', 'Avg. CPULoad.Total', 'month']]
filter1=df_cpu['month'] == mese
filtered_cpu=df_cpu[(filter1)]
filtered_cpu.head(3)


,date,hour,CSCF,Avg. CPULoad.Total,month
24,31/7/2021,23,CSCFMS03,8.320312,7
25,31/7/2021,22,CSCFMS03,9.911458,7
26,31/7/2021,21,CSCFMS03,10.973958,7


In [5]:
#elaborazione per df_bhca
day=df_bhca['Hour of DateTime'].str.split(" ").str.get(0)
month=df_bhca['Hour of DateTime'].str.split(" ").str.get(1)
year=df_bhca['Hour of DateTime'].str.split(" ").str.get(2)
df_bhca['hour']=df_bhca['Hour of DateTime'].str.split(" ").str.get(3)
d = {'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May':'5', 'June':'6', 'July':'7','August':'8','September':'9','October':'10','November':'11','December':'12'}
month = month.map(d)
df_bhca['month']=month
df_bhca['date']=day.str.cat(month, sep='/').str.cat(year, sep='/')
df_bhca=df_bhca[['date', 'hour','CSCF', 'Measure Names', 'Measure Values', 'month']]
filter1=df_bhca['month'] == mese
filtered_bhca=df_bhca[(filter1)]
filtered_bhca=filtered_bhca.groupby(['date', 'hour','CSCF']).agg({'Measure Values':'sum'})
filtered_bhca.rename(columns={'Measure Values':'bhca'}, inplace=True)
filtered_bhca.head(3)

bhca
date     hour CSCF             
1/7/2021 00   CSCFBE02  13548.0
              CSCFEH01  45097.0
              CSCFEH02  45396.0

In [6]:
#elaborazione per df_reg
day=df_reg['Minute of DateTime'].str.split(" ").str.get(0)
month=df_reg['Minute of DateTime'].str.split(" ").str.get(1)
year=df_reg['Minute of DateTime'].str.split(" ").str.get(2)
hh=df_reg['Minute of DateTime'].str.split(" ").str.get(3)
df_reg['hour']=hh.str.split(":").str.get(0)
d = {'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May':'5', 'June':'6', 'July':'7','August':'8','September':'9','October':'10','November':'11','December':'12'}
month = month.map(d)
df_reg['month']=month
df_reg['date']=day.str.cat(month, sep='/').str.cat(year, sep='/')
df_reg=df_reg[['date', 'hour','CSCF', 'Measure Names', 'Measure Values', 'month']]
filter1=df_reg['month'] == mese
filter2=df_reg['Measure Names'] == 'cscfConcurrentRegisteredUserProfiles'
filtered_reg=df_reg[(filter1)&(filter2)]
filtered_reg=filtered_reg.groupby(['date', 'hour','CSCF']).agg({'Measure Values':'max'})
filtered_reg.rename(columns={'Measure Values':'Reg'}, inplace=True)
filtered_reg.head(3)

Reg
date     hour CSCF               
1/7/2021 00   CSCFBE02   205957.0
              CSCFEH01  1177819.0
              CSCFEH02  1178592.0

In [7]:
merged_temp=filtered_reg.merge(filtered_bhca, on=['date', 'hour', 'CSCF'])
merged=merged_temp.merge(filtered_cpu, on=['date', 'hour', 'CSCF'])
merged['month']=merged['month']+'-'+year
merged['day'] = pd.to_datetime(merged['date'], format='%d/%m/%Y')
merged.head(3)

,date,hour,CSCF,Reg,bhca,Avg. CPULoad.Total,month,day
0,1/7/2021,00,CSCFBE02,205957.0,13548.0,2.695312,7-2021,2021-07-01
1,1/7/2021,00,CSCFEH01,1177819.0,45097.0,6.236979,7-2021,2021-07-01
2,1/7/2021,00,CSCFEH02,1178592.0,45396.0,6.010417,7-2021,2021-07-01


In [ ]:
#scrivo in un file ex-novo
# merged.to_excel('C:\\Users\\ccarbon\\OneDrive - Vodafone Group\\JOBFolder\\CSCF_Dashboard\\UK\\UK_CSCF_Forecast.xlsx', index=False)

In [8]:
#append in un file già esistente

filexlsx='C:\\Users\\ccarbon\\OneDrive - Vodafone Group\\JOBFolder\\CSCF_Dashboard\\UK\\UK_Forecast\\UK_CSCF_Forecast_Mar21-.xlsx'
writer = pd.ExcelWriter(filexlsx, engine='openpyxl')
# try to open an existing workbook
writer.book = load_workbook(filexlsx)
# copy existing sheets
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
# read existing file
reader = pd.read_excel(filexlsx)
# write out the new sheet
merged.to_excel(writer,index=False,header=False,startrow=len(reader)+1)
writer.close()